# ***AAAI22 MLC Paper - Baselines***

This repository contains all the instructions to reproduce the state-of-the-art models that we use as baselines in our paper. We remind you that these repositories are maintained by the authors, so if any problem arises with their code, please add an issue in the corresponding repositories.

Source codes:

1. Layered: [https://github.com/meizhiju/layered-bilstm-crf](https://github.com/meizhiju/layered-bilstm-crf)
2. Exhaustive: [https://github.com/csJd/deep_exhaustive_model](https://github.com/csJd/deep_exhaustive_model)
3. Boundary: [https://github.com/thecharm/boundary-aware-nested-ner](https://github.com/thecharm/boundary-aware-nested-ner)
4. Second-best: [https://github.com/yahshibu/nested-ner-tacl2020-flair](https://github.com/yahshibu/nested-ner-tacl2020-flair)

In [1]:
# First, we set up the working environment in google drive. If you are working locally, it will not be necessary but make sure that you are using the GPU.
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
# We will clone the repositories in the "MyDrive" folder.
%cd gdrive/MyDrive/

/content/gdrive/MyDrive


In [9]:
# We create a folder where we will clone each repository. If the folder is already created, then skip this step.
!mkdir mlc-paper-baselines

# ***Layered model***

In [ ]:
# We advance to the folder where we will save the baselines.
%cd mlc-paper-baselines/

In [ ]:
# Clone the project from the official repository. If you have already cloned it, skip this step.
!git clone https://github.com/meizhiju/layered-bilstm-crf.git

In [ ]:
# Navigate to the "src" folder. This folder contains the main scripts with which we will train and test this model.
%cd layered-bilstm-crf/src/

On the left side of Google Colab, you can navigate to the folder containing the project (MyDrive > mlc-paper-baselines > layered-bilstm-crf > src). To execute this model, we had to make the following changes to the source code:

1. **Data loading**: In the folder named "dataset", put the files train.data, dev.data and test.data. Make sure that you follow the format explained in the repository. 

2. **Embeddings**: Create a folder named "embeddings" inside the src folder, and put the embeddings described in the repository.

3. **Hyperparameters**: Change the hyperparameters and directories in the config file (add: ../src/embeddings/embedding_name to the path_pre_emb param). If you are working with the Chilean Waiting List corpus, remember that these clinical embeddings have a dimension of 300. If you are working with a GPU, set the 'main' key to 0, and if you are training, set the mode key to 'train'. As an example in the Chilean corpus:


```
- word_embedding_dim: 300 
- char_embedding_dim: 25
- dropout_ratio: 0.3
- lr_param: 0.001
- threshold: 5 
- decay_rate: 0
- batch_size: 16
- tags: 7 
- epochs: 20
- replace_digit: false
- lowercase: false
- use_singletons: false
```




4. **Code fixes**: Go to the layered_model.py script in the model folder. Due to an error generated by the class called "Evaluator", you must make the following two changes in that class:

```
1. Before (line 397): def __init__(self, iterator, target, device): 
   Now: def __init__(self, iterator, target, device=cuda.cupy):

2. Before (line 398): super(Evaluator, self).__init__(iterator=iterator, target=target, device=device) 
   Now: super(Evaluator, self).__init__(iterator=iterator, target=target)
```



In [ ]:
# We install the repository dependencies.
!pip install chainer
!pip install texttable
!pip install 'cupy-cuda101>=7.7.0,<8.0.0'

In [ ]:
# Next, we train the Layered model.
!python train.py

To evaluate, change the training mode to test in the configuration file and specify the path to the best model located in the "result" folder using the "path_model" key. In the "evaluation" folder will be found the file with the output. Then, we can calculate the task-specific metrics using this file (go to the metrics Jupyter notebook). Also, in utils.py in the "model" folder, you have to comment on the following lines to save the predictions file:



```
1. Before (line 202): os.remove(output_path)
   Now: #os.remove(output_path)

2. Before (line 203): os.remove(scores_path)
   Now: #os.remove(scores_path)
```



In [ ]:
# And we run the best model calculated on the validation set, now on our testing data. The file with the predictions will be located in the "evaluation" folder with the extension '.scores'.
!python test.py

# ***Exhaustive model***

In [ ]:
# If you were in the layered folder
#%cd ..
#%cd ..

In [ ]:
# We advance to the folder where we will save the baselines.
%cd mlc-paper-baselines/

In [ ]:
# Clone the project from the official repository. If you have already cloned it, skip this step.
!git clone https://github.com/csJd/deep_exhaustive_model.git

In [ ]:
# In the folder called "data", we will create a folder for each dataset. We also create a folder for the models.
%cd deep_exhaustive_model/data
!mkdir genia
!mkdir wl
!mkdir germ
!mkdir model
%cd ..

On the left side of Google Colab, you can navigate to the folder containing the project (MyDrive > mlc-paper-baselines > deep_exhaustive_model). To execute this model, we had to make the following changes to the source code:

For example using the Chilean Waiting List dataset.

1. **Data loading**: In the folder named "data", wl.put the files train.iob2, wl.dev.iob2 and wl.test.iob2 in the wl folder, following the format explained in the repository (As an example, we will use the case of the waiting list).

2. **Embeddings**: Put the embeddings described in the repository in the "embedding" folder, which is inside the "data" folder.

3. **Hyperparameters**: Change the hyperparameters and directories in the "train.py" script. If you are working with the Chilean Waiting List corpus, remember that these clinical embeddings have a dimension of 300 (line 90). As an example in the Chilean corpus:



```
- embedding_dim: 300 
- char_feat_dim: 25
- learning_rate: 0.001
- clip_norm: 5 
- batch_size: 16
- epochs: 30
- hidden_size = 128
```

4. **Code fixes**: You must make the following changes in lines 24 to 28.

```
EMBD_URL = from_project_root("data/embedding/cwlce.vec")
VOCAB_URL = from_project_root("data/wl/vocab.json")
TRAIN_URL = from_project_root("data/wl/wl.train.iob2")
DEV_URL = from_project_root("data/wl/wl.dev.iob2")
TEST_URL = from_project_root("data/wl/wl.test.iob2")
```

In the "dataset.py script", you have to change "data_urls" (line 272) variable to: 

```
data_urls = [from_project_root("data/wl/wl.train.iob2"),
                 from_project_root("data/wl/wl.dev.iob2"),
                 from_project_root("data/wl/wl.test.iob2")]
```

Add the pre-trained embedding path (line 275)
```
prepare_vocab(data_urls, from_project_root("data/embedding/cwlce.vec"), update=True, min_count=1)
```

Finally, in "model.py", change the CharLSTM class (line 101), use the following:


```
class CharLSTM(nn.Module):

    def __init__(self, n_chars, embedding_size, hidden_size, lstm_layers=1, bidirectional=True):
        super().__init__()
        self.n_chars = n_chars
        self.embedding_size = embedding_size
        self.n_hidden = hidden_size * (1 + bidirectional)

        self.embedding = nn.Embedding(n_chars, embedding_size, padding_idx=0)

        self.lstm = nn.LSTM(
            input_size=embedding_size,
            hidden_size=hidden_size,
            bidirectional=bidirectional,
            num_layers=lstm_layers,
            batch_first=True,
        )

    def sent_forward(self, words, lengths, indices):
        sent_len = words.shape[0]
        # words shape: (sent_len, max_word_len)

        embedded = self.embedding(words)
        # in_data shape: (sent_len, max_word_len, embedding_dim)

        packed = nn.utils.rnn.pack_padded_sequence(embedded, lengths.cpu().numpy(), batch_first=True)
        _, (hn, _) = self.lstm(packed)
        # shape of hn:  (n_layers * n_directions, sent_len, hidden_size)

        hn = hn.permute(1, 0, 2).contiguous().view(sent_len, -1)
        hn2 = hn.clone()
        # shape of hn:  (sent_len, n_layers * n_directions * hidden_size) = (sent_len, 2*hidden_size)

        # shape of indices: (sent_len, max_word_len)
        hn[indices] = hn2  # unsort hn
        # unsorted = hn.new_empty(hn.size())
        # unsorted.scatter_(dim=0, index=indices.unsqueeze(-1).expand_as(hn), src=hn)
        return hn

    def forward(self, sentence_words, sentence_word_lengths, sentence_word_indices):
        # sentence_words [batch_size, *sent_len, max_word_len]
        # sentence_word_lengths [batch_size, *sent_len]
        # sentence_word_indices [batch_size, *sent_len, max_word_len]

        batch_size = len(sentence_words)
        batch_char_feat = torch.nn.utils.rnn.pad_sequence(
            [self.sent_forward(sentence_words[i], sentence_word_lengths[i], sentence_word_indices[i])
             for i in range(batch_size)], batch_first=True)

        return batch_char_feat
        # (batch_size, sent_len, 2 * hidden_size)
```






In [ ]:
# Now, we create the dataset pickle files.
!python dataset.py

In [ ]:
# And we train the Exhaustive model.
!python train.py

To evaluate and generate the prediction file, in the eval.py script, uncomment the following line (line 144): 

```
# predict_on_iob2(model, test_url)
```

Change the model_url to the best model found (line 139):

```
model_url = from_project_root("data/model/model.pt")
```

And, change the test_url (line 143):


```
test_url = from_project_root("data/wl/wl.test.iob2")
```

In [25]:
!python eval.py

In the corpus folder (wl in this example) will be located the wl.test.pred.txt file with the output, to which the metrics can be calculated in our notebook of task-specific metrics. Note that in this output, the multilabel entities are not considered. Therefore, the Chilean Waiting List predictions must be compared with the original data to obtain the objective metric. This is done in the metrics notebook.

# ***Boundary model***

In [28]:
# If you were in the exhaustive folder
# %cd ..
# %cd ..

/content/gdrive/My Drive


In [ ]:
# Clone the project from the official repository. If you have already cloned it, skip this step.
%cd mlc-paper-baselines/

In [ ]:
# Clone the project from the unofficial repository, if you have already cloned it skip this step.
!git clone https://github.com/thecharm/boundary-aware-nested-ner

In [33]:
%cd boundary-aware-nested-ner/Our_boundary-aware_model/

/content/gdrive/My Drive/mlc-paper-baselines/boundary-aware-nested-ner/Our_boundary-aware_model


In [34]:
# In the folder called "data", we will create a folder for each dataset. We also create a folder for the models and the embeddings folder.
%cd data
!mkdir wl
!mkdir germ
!mkdir model
!mkdir embedding
%cd ..

/content/gdrive/My Drive/mlc-paper-baselines/boundary-aware-nested-ner/Our_boundary-aware_model/data
/content/gdrive/My Drive/mlc-paper-baselines/boundary-aware-nested-ner/Our_boundary-aware_model


On the left side of Google Colab, you can navigate to the folder containing the project (MyDrive > mlc-paper-baselines > boundary-aware-nested-ner > Our_boundary-aware_model). To execute this model, we had to make the following changes to the source code:

For example using the Chilean Waiting List dataset.

1. **Data loading**: In the "data" folder, place the files wl.train.iob2, wl.dev.iob2 and wl.test.iob2 in the wl folder, following the format explained in the repository. (As an example, we will use the case of the waiting list).

2. **Embeddings**: Put the embeddings described in the repository in the "embedding" folder, which is inside the "data" folder.

3. **Hyperparameters**: Change the hyperparameters and directories in the "train.py" script. If you are working with the Chilean Waiting List corpus, remember that these clinical embeddings have a dimension of 300 (line 95). 



```
MAX_REGION = 10
EARLY_STOP = 5
LR = 0.005
BATCH_SIZE = 16 
MAX_GRAD_NORM = 5
N_TAGS = 8
TAG_WEIGHTS = [1, 1, 1, 1, 1, 1, 1, 1]
FREEZE_WV = False
LOG_PER_BATCH = 10
hidden_size=128,
lstm_layers=3
```


4. **Code fixes**

Lines 34-38

```
PRETRAINED_URL = from_project_root("data/embedding/cwlce.vec")
EMBED_URL = from_project_root("data/wl/embeddings.npy")
VOCAB_URL = from_project_root("data/wl/vocab.json")
TRAIN_URL = from_project_root("data/wl/wl.train.iob2")
DEV_URL = from_project_root("data/wl/wl.dev.iob2")
TEST_URL = from_project_root("data/wl/wl.test.iob2")
```

Change tags and weights
```
N_TAGS = 8 (line 29)
TAG_WEIGHTS = [1, 1, 1, 1, 1, 1, 1, 1] (line 30)
```

And delete the following lines:
```
import pdb (line 21)
pdb.set_trace() (line 22)
```

In the "dataset.py" script, change data_urls (line 286) to 

```
data_urls = [from_project_root("data/wl/wl.train.iob2"),
                 from_project_root("data/wl/wl.dev.iob2"),
                 from_project_root("data/wl/wl.test.iob2")]
```

Add the pre-trained embedding path (line 289)
```
prepare_vocab(data_urls, from_project_root("data/embedding/cwlce.vec"), update=True, min_count=1)
```

Change these three variables (lines 15-17):
```
LABEL_IDS = {"neither": 0, "Disease": 1, "Finding": 2, "Medication": 3, "Procedure": 4, "Family_Member": 5,"Body_Part": 6, "Abbreviation": 7}
PRETRAINED_URL = from_project_root("data/embedding/cwlce.vec")
LABEL_LIST = {"O", "Disease", "Finding", "Medication", "Procedure", "Family_Member","Body_Part", "Abbreviation"}
```

Finally, in "model.py", change the CharLSTM class, use the following:

```
class CharLSTM(nn.Module):

    def __init__(self, n_chars, embedding_size, hidden_size, lstm_layers=1, bidirectional=True):
        super().__init__()
        self.n_chars = n_chars
        self.embedding_size = embedding_size
        self.n_hidden = hidden_size * (1 + bidirectional)

        self.embedding = nn.Embedding(n_chars, embedding_size, padding_idx=0)

        self.lstm = nn.LSTM(
            input_size=embedding_size,
            hidden_size=hidden_size,
            bidirectional=bidirectional,
            num_layers=lstm_layers,
            batch_first=True,
        )

    def sent_forward(self, words, lengths, indices):
        sent_len = words.shape[0]
        # words shape: (sent_len, max_word_len)

        embedded = self.embedding(words)
        # in_data shape: (sent_len, max_word_len, embedding_dim)

        packed = nn.utils.rnn.pack_padded_sequence(embedded, lengths.cpu().numpy(), batch_first=True)
        _, (hn, _) = self.lstm(packed)
        # shape of hn:  (n_layers * n_directions, sent_len, hidden_size)

        hn = hn.permute(1, 0, 2).contiguous().view(sent_len, -1)
        hn2 = hn.clone()
        # shape of hn:  (sent_len, n_layers * n_directions * hidden_size) = (sent_len, 2*hidden_size)

        # shape of indices: (sent_len, max_word_len)
        hn[indices] = hn2  # unsort hn
        # unsorted = hn.new_empty(hn.size())
        # unsorted.scatter_(dim=0, index=indices.unsqueeze(-1).expand_as(hn), src=hn)
        return hn

    def forward(self, sentence_words, sentence_word_lengths, sentence_word_indices):
        # sentence_words [batch_size, *sent_len, max_word_len]
        # sentence_word_lengths [batch_size, *sent_len]
        # sentence_word_indices [batch_size, *sent_len, max_word_len]

        batch_size = len(sentence_words)
        batch_char_feat = torch.nn.utils.rnn.pad_sequence(
            [self.sent_forward(sentence_words[i], sentence_word_lengths[i], sentence_word_indices[i])
             for i in range(batch_size)], batch_first=True)

        return batch_char_feat
        # (batch_size, sent_len, 2 * hidden_size)
```

In [35]:
!python train.py

generating vocab from ['/content/gdrive/My Drive/mlc-paper-baselines/boundary-aware-nested-ner/Our_boundary-aware_model/data/wl/wl.train.iob2', '/content/gdrive/My Drive/mlc-paper-baselines/boundary-aware-nested-ner/Our_boundary-aware_model/data/wl/wl.dev.iob2', '/content/gdrive/My Drive/mlc-paper-baselines/boundary-aware-nested-ner/Our_boundary-aware_model/data/wl/wl.test.iob2']
generating pre-trained embedding from /content/gdrive/My Drive/mlc-paper-baselines/boundary-aware-nested-ner/Our_boundary-aware_model/data/embedding/cwlce.vec
arguments {
  "n_epochs": 30,
  "embedding_url": "/content/gdrive/My Drive/mlc-paper-baselines/boundary-aware-nested-ner/Our_boundary-aware_model/data/wl/embeddings.npy",
  "char_feat_dim": 50,
  "freeze": false,
  "train_url": "/content/gdrive/My Drive/mlc-paper-baselines/boundary-aware-nested-ner/Our_boundary-aware_model/data/wl/wl.train.iob2",
  "dev_url": "/content/gdrive/My Drive/mlc-paper-baselines/boundary-aware-nested-ner/Our_boundary-aware_model

Replace eval.py with the one we passed in the supplementary material. This edited file writes the result of the predictions in a text file, which can be used to calculate task-specific metrics. The name of the file is boundary_result.txt. Also, change the following lines, according to your dataset (just in testing step). In the "model" folder you will find the best model save.

```
model_url = from_project_root("data/model/end2end_model_epochnumber_score.pt")
test_url = from_project_root("data/wl/wl.test.iob2")
```

In [ ]:
!python eval.py

# ***Second-best model***

In [ ]:
%cd mlc-baselines/

/content/gdrive/My Drive/mlc-baselines


In [ ]:
# Clone the project from the unofficial repository, if you have already cloned it skip this step.
!git clone https://github.com/yahshibu/nested-ner-tacl2020-flair.git

Cloning into 'nested-ner-tacl2020-flair'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 45 (delta 9), reused 45 (delta 9), pack-reused 0
Unpacking objects: 100% (45/45), done.


In [ ]:
%cd nested-ner-tacl2020-flair/

/content/gdrive/My Drive/mlc-baselines/nested-ner-tacl2020-flair


In [ ]:
!mkdir embeddings

Place the embeddings described in the repository in the embeddings folder.

In [ ]:
!pip install adabound
!pip install flair

Put in the data/wl folder the three files wl.train, wl.dev, wl.test.

Add to the folder the files gen_data_for_wl.py and gen_data_for_germ.py

Change the reader file to the one we deliver.

Change the embeddings contextualized in the sequence_labeling.py file to those described in the repository

Change the hyperparameters in the configuration file and links.

In [ ]:
!python gen_data_for_wl.py

Max length: 42
Threshold 6: 2734
Max length: 35
Threshold 6: 277
Max length: 29
Threshold 6: 397
# mentions: 35480
# mentions: 3971
# mentions: 4289
All 35480, start 0, end 0
All 3971, start 0, end 0
All 4289, start 0, end 0
Remember to scp word vectors as well


Module - crf - change device problem: indices_3 = indices_3.cuda() before "while t greater than -1"

Add W2Vebedding class to sequence labeling....

In [ ]:
!python train.py

2021-08-31 22:58:24,684 - Nested Mention - INFO - {'root_path': '.', 'data_set': 'wl', 'batch_size': 8, 'if_shuffle': True, 'label_size': 7, 'char_embed': 32, 'num_filters': 32, 'word_dropout': 0.3, 'hidden_size': 256, 'layers': 2, 'lstm_dropout': 0.5, 'embed_path': './data/word_vec_wl.vec', 'epoch': 1, 'if_gpu': True, 'opt': <Optimizer.SGD: 'SGD'>, 'lr': 0.1, 'final_lr': None, 'l2': 0.0, 'check_every': 1, 'clip_norm': 5, 'lr_patience': 5, 'data_path': './data/wl', 'train_data_path': './data/wl_train.pkl', 'dev_data_path': './data/wl_dev.pkl', 'test_data_path': './data/wl_test.pkl', 'config_data_path': './data/wl_config.pkl', 'model_root_path': './dumps', 'model_path': './dumps/wl_model'}
2021-08-31 22:58:45,267 - Nested Mention - INFO - 251 batches expected for training
2021-08-31 22:58:45,267 - Nested Mention - INFO - 
2021-08-31 22:58:45,268 - Nested Mention - INFO - Epoch 1 (learning rate=0.1000):
/content/gdrive/MyDrive/mlc-baselines/nested-ner-tacl2020-flair/model/sequence_labeli